# Holt-Winters Metrics

This notebook fits a multiplicative Holt-Winters model for various combinations of alpha, gamma, delta values ranging from 0.0 to 1.0 and save evaluation results.

In [1]:
import module.util_functions as utf
import module.constants as const
from tqdm import tqdm
import os
import pandas as pd
import numpy as np

# load sales data for all stores
store_sales = utf.load_ts_data('store_sales.csv')

In [2]:
# set random state
np.random.seed(45)

max_iters = 1500      # set maximum number of iterations
parms_list = []       # a list of [alpha, gamma, delta]

# randomly generate a list of combinations of alpha, delta, gamma without replacement
for i in range(0, max_iters):
    while True:
        alpha = np.round(np.random.uniform(0, 1), 2)
        gamma = np.round(np.random.uniform(0, 1), 2)
        delta = np.round(np.random.uniform(0, 1), 2)
        parms = [alpha, gamma, delta]
        if parms not in parms_list:
            break
            
    parms_list.append(parms)

In [3]:
from tqdm import tqdm
from module.holt_winters import HoltWinters

store_list = []               # a list of stores
alpha_list = []               # a list of alpha values
gamma_list = []               # a list of gamma values
delta_list = []               # a list of delta values
mad_list = []                 # a list of MAD
mape_list = []                # a list of MAPE
rmse_list = []                # a list of RMSE
r_squared_list = []           # a list of R-Squared
train_times = []              # a list of model's training time

# iterate through every store
for store_num in store_sales.Store.unique():
    print('Store: %d' % store_num)
    data = utf.get_store_sales(store_sales, store_num)    # get sales data for a given store
    train = data[:-const.TEST_SIZE]                       # set train dataset
    preload = None                                        # store preload model from previous run
    
    # evaluate model for different combinations of alpha, gamma, and delta
    for i in tqdm(range(0, max_iters)):
        alpha = parms_list[i][0]
        gamma = parms_list[i][1]
        delta = parms_list[i][2]
        
        # train Holt-Winters model and make sales forecasts for the next 52 weeks
        model = HoltWinters(alpha, gamma, delta, preload=preload)
        model.fit(data[:-const.TEST_SIZE])
        forecasts = model.predict()
            
        if preload is None:
            # set preload model to reduce training time for successive models that use the same set of train data
            preload = model

        # evaluate model and save result
        mape, mad, rmse, r_squared = utf.compute_metrics(data.tail(const.TEST_SIZE)[const.STORE_OBSERVE].values, 
                                                         forecasts[model.forecast_test_label].values)
        mape_list.append(mape)
        mad_list.append(mad)
        rmse_list.append(rmse)
        r_squared_list.append(r_squared)
        train_times.append(model.train_time)
        alpha_list.append(alpha)
        gamma_list.append(gamma)
        delta_list.append(delta)
        store_list.append(store_num)

Store: 1


100%|██████████| 1500/1500 [05:37<00:00,  4.45it/s]


Store: 2


100%|██████████| 1500/1500 [05:44<00:00,  4.35it/s]


Store: 3


...

100%|██████████| 1500/1500 [05:25<00:00,  4.61it/s]


Store: 44


100%|██████████| 1500/1500 [05:19<00:00,  4.70it/s]


Store: 45


100%|██████████| 1500/1500 [05:04<00:00,  4.92it/s]


In [4]:
# create a metric dataframe and save as a csv file
metrics = pd.DataFrame({'Store': store_list, 'Alpha': alpha_list, 'Gamma': gamma_list, 'Delta': delta_list,
                        'MAPE': mape_list, 'MAD': mad_list, 'RMSE': rmse_list, 'R-Squared': r_squared_list,
                        'Train Time': train_times})
metrics['Model'] = ["Holt-Winters"] * metrics.shape[0]
metrics.to_csv(os.path.join('results', 'hw_metrics.csv'), index=False)